In [1]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO 
import pandas as pd
import matplotlib.pyplot as plt

# Import the sequence(s)

In [16]:
seq_record = SeqIO.read('sequences/test/ELF3.fa','fasta')
#seq_record.id 
seq_record.seq 

Seq('MKRGKDEEKILEPMFPRLHVNDADKGGPRAPPRNKMALYEQLSIPSQRFGDHGT...SKP', SingleLetterAlphabet())

## Multiple sequences

In [2]:
seq = list(SeqIO.parse('sequences/test/sequences.fa','fasta'))
seq[0].seq

Seq('TGGATATTCTGTGAGTTTGGCCCTAGCT', SingleLetterAlphabet())

# Call blast from the NCBI website

In [5]:
result_handle = []
for i in range(50):
    result_handle.append(NCBIWWW.qblast("blastn", "nt", seq[i].seq))

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [22]:
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None)
    BLAST search using NCBI's

In [21]:
from Bio import Entrez
Entrez.email = 'ness.louafi@gmail.com'
counter = 0 
list_record_host = []
for record in SeqIO.parse("sequences/test/sequences.fa", format="fasta"):
    print(record.id)
    print(counter)
    counter+=1
#         print(record.seq)

    # online request
    try:
        result_handle = NCBIWWW.qblast("blastn","nt", record.format("fasta"),record.seq)
        print(result_handle)
    except HTTPError:
        time.sleep(5)
        result_handle = NCBIWWW.qblast("blastn","nt", record.format("fasta"),record.seq)

    # result handle stored in a list
    list_record_host.append(result_handle)

0
0


NameError: name 'HTTPError' is not defined

# Save and open the results

In [52]:
for i in range(50):
    with open('results'+str(i)+'.xml', 'w') as save_file: 
        blast_results = result_handle.read() 
        save_file.write(blast_results)

E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("results.xml")): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH: 
                    print("match: %s " % align.title[:100])



query: unnamed protein product
match: ref|NP_180164.1| hydroxyproline-rich glycoprotein family protein [Arabidopsis thaliana] >sp|O82804.1 
match: emb|VYS53527.1| unnamed protein product [Arabidopsis thaliana] 
match: gb|KAG7642078.1| hypothetical protein ISN44_As02g020390 [Arabidopsis suecica] 
match: gb|KAG7637467.1| hypothetical protein ISN45_At02g019970 [Arabidopsis thaliana x Arabidopsis arenosa] 
match: emb|CAA0371864.1| unnamed protein product [Arabidopsis thaliana] 
match: emb|CAA72719.1| nematode responsive protein [Arabidopsis thaliana] 
match: emb|CAD5319528.1| unnamed protein product [Arabidopsis thaliana] 
match: gb|OAP09466.1| PYK20 [Arabidopsis thaliana] 
match: gb|EFH55167.1| hypothetical protein ARALYDRAFT_481449 [Arabidopsis lyrata subsp. lyrata] 
match: emb|CAE6010521.1| unnamed protein product [Arabidopsis arenosa] 
match: gb|KAG7572294.1| hypothetical protein ISN44_As09g006620 [Arabidopsis suecica] 
match: ref|XP_020883196.1| protein EARLY FLOWERING 3 isoform X2 

In [13]:
blast_params = {'program': 'blastp', 'database': 'nr', 'sequence': seq_record.seq, 'expect': 10.0}
blast_params['database'] = ['nr']
print_data = pd.DataFrame()
for database in blast_params['database']:
    db_values = {}
    result = NCBIWWW.qblast(blast_params['program'], database, blast_params['sequence'], expect=blast_params['expect'])
    file_name = "blast_output_" + database + ".xml"
    with open(file_name, "w") as output_xml:
        output_xml.write(result.read())
    result.close()
    result_input = open(file_name)
    blast_records = NCBIXML.read(result_input)
    for description in blast_records.descriptions:
        if 'score' in db_values:
            db_values['score'].append(description.score)
        else:
            db_values['score'] = [description.score]
        if 'e-value' in db_values:
            db_values['e-value'].append(description.e)
        else:
            db_values['e-value'] = [description.e]
    df = pd.DataFrame.from_dict(db_values)
    df['database'] = database[0:6] # we simply limit the name to the first 6 characters for easier viewing
    frames = [print_data, df]
    print_data = pd.concat(frames, ignore_index=True)
    

In [26]:
E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("blast_output_nr.xml")): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH:
                    if "Arabidopsis thaliana" in align.title:
                        print("match: %s " % align.title[:100])



query: unnamed protein product
match: ref|NP_180164.1| hydroxyproline-rich glycoprotein family protein [Arabidopsis thaliana] >sp|O82804.1 
match: emb|VYS53527.1| unnamed protein product [Arabidopsis thaliana] 
match: gb|KAG7637467.1| hypothetical protein ISN45_At02g019970 [Arabidopsis thaliana x Arabidopsis arenosa] 
match: emb|CAA0371864.1| unnamed protein product [Arabidopsis thaliana] 
match: emb|CAA72719.1| nematode responsive protein [Arabidopsis thaliana] 
match: emb|CAD5319528.1| unnamed protein product [Arabidopsis thaliana] 
match: gb|OAP09466.1| PYK20 [Arabidopsis thaliana] 
match: gb|KAG7567818.1| hypothetical protein ISN45_Aa04g006560 [Arabidopsis thaliana x Arabidopsis arenosa] 
match: gb|AAM97051.1| unknown protein [Arabidopsis thaliana] >gb|AAN15504.1| unknown protein [Arabidopsis t 
match: gb|OAP09467.1| PYK20 [Arabidopsis thaliana] 


# Open when finished

In [ ]:
import webbrowser
webbrowser.open_new('http://localhost:8888/notebooks/Documents/iGEM/iGEM/Run%20BLAST.ipynb')